In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece
!pip install torch
!pip install transformers


     |████████████████████████████████| 1.2MB 5.6MB/s 
     |████████████████████████████████| 1.8MB 5.4MB/s 
     |████████████████████████████████| 890kB 20.9MB/s 
     |████████████████████████████████| 2.9MB 22.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5c3845d8d2585d61589837250064cf865a4d93a58abadf94cd9089a9fd437a75
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#import pretrained models 
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
#model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-tw")

In [ ]:
#import further libraries that will be needed
import nltk
from nltk.translate.bleu_score import SmoothingFunction
import pandas as pd

In [ ]:
def translate(input_sentence, model=model, **kwargs):
    """
    Translate the sentence to the target language using the provided model and any keyword arguments provided.
    input_sentence: List of sentences
    model: model to use for the translation.
    
    Returns:
    List of corresponding translations for the target language
    """

    input_id = tokenizer.encode(input_sentence, return_tensors="pt")
    beam_output = model.generate(input_id,  **kwargs)
    return tokenizer.decode(beam_output[0], skip_special_tokens=True)

In [ ]:
def bleuscore_dataframe(dataframe, weights=(0.85,0.15,0,0), smoothFn=None,auto_reweigh=False):
    """
    compute the bleu score of an entire dataframe dataframe should have a predictions and 
    references column
    dataframe: pd.DataFrame - should have "predictions" and "references" columns
    weights is a 4-tuple summing up to 1. indicates weights for unigrams, bigrams etc
    smoothFn:int {0,7}. refer to nltk API for details
    """
    preds = []
    for line in dataframe['predictions'].values.tolist(): 
        line_clean = line.strip().split()
        preds.append(line_clean)
    refs = []
    for line in dataframe["references"].values.tolist(): 
        line_clean = str(line).strip().split()
        refs.append(line_clean)
    refs_2 = [list([(item)]) for item in refs]  # putting it in a form for the nltk bleu method
    if smoothFn in range(8):
        smoothFunc = SmoothingFunction()
        smoothFunc = eval(f"smoothFunc.method{smoothFn}")
    else:
        smoothFunc = smoothFn
    BLEUscore = nltk.translate.bleu_score.corpus_bleu(refs_2, preds, weights=weights,
                                                      smoothing_function=smoothFunc,auto_reweigh=auto_reweigh)
    return BLEUscore

In [ ]:
#read the parrarel dataset to be used for checking scores
pairs = pd.read_csv("/content/drive/My Drive/GhanaNLP/DATA/GhanaNLP_data_001.csv")

In [ ]:
def predict_sample_sentences(parallel_sentences, sample_size=100):
  """
    Takes a dataframe of parrarel eng-twi sentences and translate the english sentences to target language


    parallel_sentences: pd.DataFrame - should have "English" and "references" columns with "English" sentences in column index 0 and "references in col index 1
    sample_size : number of sentences in the dataframe to translate, default is 100 sentences
    
    Return:
    Returns a dataframe with a third column "Predictions" added which contain the translated sentences
    """

  test_pairs = parallel_sentences.sample(sample_size)
  test_pairs.columns= ["English", "references"]
  test_pairs["predictions"] = test_pairs["English"].apply(translate)
  return test_pairs

In [ ]:
#translate some sample sentences
sample_test_pairs = predict_sample_sentences(pairs)

In [ ]:
#take a preview of 6 of the sentences
sample_test_pairs.sample(6, replace=False)

,English,references,predictions
619,Pokua must consider carefully whether he reall...,Ɛsɛ sɛ Pokua dwene awaregyae no ho yiye,Ɛsɛ sɛ Pokua susuw ho yiye hwɛ sɛ ebia ɔpɛ awa...
578,I'm practically already an adult.,Manyini awie ara nen,Ɛkame ayɛ sɛ meyɛ ɔpanyin dedaw.
640,I want those back when you're done.,Wowie a fabrɛ me,"Sɛ woreyɛ biribi a, mepɛ sɛ wɔn a wɔwɔ hɔ no y..."
482,That is a pencil.,ɛno yɛ twerɛdua,Ɛno yɛ pɛnsere.
572,The boy tried to be a man and not cry.,Abaamuwaa no bɔɔ ne ho mmɔden sɛ ɔrensu,"Abarimaa no bɔɔ mmɔden sɛ ɔbɛyɛ ɔbarima, na ɛn..."
47,He has leukaemia.,Ɔwɔ mogya mu kokram,Ɔwɔ leukaemia.


In [18]:
#check some sample bluescores with different paramaters
#1. default bleu score weights on 4-grams weights=(0.25,0.25,0.25,0.25) and no smoothing function and no auto_reweigh - Option to re-normalize the weights uniformly.
bleuscore_dataframe(sample_test_pairs[["references","predictions"]])

0.2558354187605164

In [ ]:
#3. put all emphasis on uni-grams (1.0)  and smoothing function of 7 and with auto_reweigh
bleuscore_dataframe(sample_test_pairs[["references","predictions"]], weights=(1,0,0,0),smoothFn=7, auto_reweigh=True)

0.5687161015194799

In [ ]:
#3. default bleu score weights on 4-grams weights=(0.25,0.25,0.25,0.25)  and smoothing function of 7 and with auto_reweigh
bleuscore_dataframe(sample_test_pairs[["references","predictions"]],smoothFn=7, auto_reweigh=True)

0.5038560355183354